<a href="https://colab.research.google.com/github/havehill/BOAZ_20anal_mini_prj/blob/EDA/Derived_Variable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

df_raw = pd.read_csv('/content/drive/MyDrive/Brazilian_Customer.csv')
print("결측치 확인")
print(df_raw.isnull().sum())

결측치 확인
order_id                              0
customer_id                           0
order_status                          0
order_purchase_timestamp              0
order_approved_at                    15
order_delivered_carrier_date       1235
order_delivered_customer_date      2471
order_estimated_delivery_date         0
review_id                             0
review_score                          0
review_comment_title             103437
review_comment_message            67650
review_creation_date                  0
review_answer_timestamp               0
payment_sequential                    0
payment_type                          0
payment_installments                  0
payment_value                         0
customer_unique_id                    0
customer_zip_code_prefix              0
customer_city                         0
customer_state                        0
order_item_id                         0
product_id                            0
seller_id                        

In [3]:
# 결측치 제거
df_raw = df_raw.drop(['review_comment_title'], axis = 1)
df_raw = df_raw.dropna(axis = 0)

In [4]:
df = df_raw.copy()

#### 배송 관련 파생변수

In [5]:
df["order_purchase_timestamp"] = pd.to_datetime(df_raw["order_purchase_timestamp"], format='%Y-%m-%d %H:%M:%S')
df["order_delivered_customer_date"] = pd.to_datetime(df_raw["order_delivered_customer_date"], format='%Y-%m-%d %H:%M:%S')
df["order_estimated_delivery_date"] = pd.to_datetime(df_raw["order_estimated_delivery_date"], format='%Y-%m-%d %H:%M:%S')

In [6]:
# 배송 기간 변수 : delivery_period

df['delivery_period'] = df['order_purchase_timestamp'] - df['order_delivered_customer_date']

In [7]:
# 조기 배송 변수 : early_delivery

df['early_delivery'] = df['order_estimated_delivery_date'] - df['order_delivered_customer_date']

In [8]:
import numpy as np

early_delivery_div = []

for d in df['early_delivery']:
  if(d.days < 0) :
    early_delivery_div.append(-1)
  elif (d.days == 0) :
    early_delivery_div.append(0)
  else :
    early_delivery_div.append(1)

df['early_delivery_div'] = early_delivery_div

In [9]:
# 조기 배송 여부 변수 : early_delivery_div

df['early_delivery_div'].value_counts()

 1    41874
-1     4874
 0      621
Name: early_delivery_div, dtype: int64

#### 가격 관련 파생 변수

#####소득 수준 변수

In [10]:
income_level = []

for p in df['payment_installments']:
  if(p >= 0 and p < 8):
    income_level.append(2)
  elif(p >= 8 and p <16):
    income_level.append(1)
  else:
    income_level.append(0)
  
df['income_level'] = income_level

In [11]:
df['income_level'].value_counts(0)

2    41342
1     5964
0       63
Name: income_level, dtype: int64

##### 카테고리별 가격 변수 : cat_cheap

In [12]:
cat = df.groupby(['product_category_name'])

In [13]:
cat = pd.DataFrame(cat['price'].quantile(0.25))

In [14]:
cat = cat.rename(columns = {'price':'price_0.25'})

In [15]:
df = pd.merge(df,cat, how='left',on='product_category_name')

In [16]:
df['cat_cheap'] = 0
df.loc[df['price'] < df['price_0.25'], 'cat_cheap'] = 1

In [17]:
df.cat_cheap.value_counts()

0    35843
1    11526
Name: cat_cheap, dtype: int64

In [18]:
df = df.drop(['price_0.25'], axis = 1)

#### 지역 관련 파생변수

##### 지역 구분 : cu_region

In [19]:
df.loc[df[df['customer_state'].isin(['RR','PA','AP','AM','TO','AC','RO'])].index, 'cu_region'] = 'north'
df.loc[df[df['customer_state'].isin(['MA','PE','PI','CE','RN','PB','AL','SE','BA'])].index, 'cu_region'] = 'north east'
df.loc[df[df['customer_state'].isin(['MT','GO','DF','MS'])].index, 'cu_region'] = 'mid west'
df.loc[df[df['customer_state'].isin(['MG','ES','RJ','SP'])].index, 'cu_region'] = 'south east'
df.loc[df[df['customer_state'].isin(['PR','SC','RS'])].index, 'cu_region'] = 'south'

In [20]:
df.cu_region.value_counts()

south east    32183
south          6264
north east     5164
mid west       2805
north           953
Name: cu_region, dtype: int64

##### 발전도 낮은 지역 파생변수 : low_develop

In [21]:
df['low_develop'] = 0
df.loc[df[df.cu_region.isin(['north','north east'])].index, 'low_develop'] = 1

In [22]:
df.low_develop.value_counts()

0    41252
1     6117
Name: low_develop, dtype: int64

#### 카테고리 파생변수

In [23]:
# 카테고리 대분류 변수 : 분류 기준은 노션 > 파생변수 생성 > 카테고리 분류 참고!

def switch(x):
  return {
      'cama_mesa_banho' : 0,
      'beleza_saude' : 8,
      'esporte_lazer' : 8,
      'moveis_decoracao' : 3,
      'informatica_acessorios' : 4,
      'utilidades_domesticas' : 2,
      'relogios_presentes' : 6,
      'telefonia' : 4,
      'ferramentas_jardim' : 2,
      'automotivo' : 5,
      'brinquedos' : 6,
      'cool_stuff' : 0,
      'perfumaria' : 0,
      'bebes' : 1,
      'eletronicos' : 4,
      'papelaria' : 6,
      'fashion_bolsas_e_acessorios' : 0,
      'pet_shop' : 7,
      'moveis_escritorio' : 3,
      'consoles_games' : 4,
      'malas_acessorios' : 0,
      'construcao_ferramentas_construcao' : 3,
      'eletrodomesticos' : 4,
      'instrumentos_musicais' : 6,
      'eletroportateis' : 4,
      'casa_construcao' : 3,
      'livros_interesse_geral' : 6,
      'moveis_sala' : 3,
      'alimentos' : 2,
      'casa_conforto' : 2,
      'bebidas' : 2,
      'audio' : 4,
      'market_place' : 9,
      'construcao_ferramentas_iluminacao' : 3,
      'moveis_cozinha_area_de_servico_jantar_e_jardim' : 3,
      'climatizacao' : 4,
      'alimentos_bebidas' : 2,
      'fashion_calcados' : 0,
      'telefonia_fixa' : 4,
      'industria_comercio_e_negocios' : 9,
      'livros_tecnicos' : 6,
      'eletrodomesticos_2' : 4,
      'agro_industria_e_comercio' : 9,
      'construcao_ferramentas_jardim' : 9,
      'artes' : 0,
      'pcs' : 9,
      'sinalizacao_e_seguranca' : 9,
      'construcao_ferramentas_seguranca' : 3,
      'artigos_de_natal' : 0,
      'fashion_roupa_masculina' : 0,
      'fashion_underwear_e_moda_praia' : 0,
      'moveis_quarto' : 3,
      'construcao_ferramentas_ferramentas' : 3,
      'tablets_impressao_imagem' : 4,
      'portateis_casa_forno_e_cafe' : 2,
      'cine_foto' : 6,
      'dvds_blu_ray' : 6,
      'livros_importados' : 6,
      'fashion_roupa_feminina' : 0,
      'artigos_de_festas' : 2,
      'moveis_colchao_e_estofado' : 2,
      'musica' : 9,
      'fraldas_higiene' : 2,
      'fashion_esporte' : 0,
      'flores' : 2,
      'casa_conforto_2' : 2,
      'artes_e_artesanato' : 0,
      'portateis_cozinha_e_preparadores_de_alimentos' : 2,
      'la_cuisine' : 2,
      'cds_dvds_musicais' : 6,
      'pc_gamer' : 4,
      'fashion_roupa_infanto_juvenil' : 0,
      'seguros_e_servicos' : 9
  }.get(x, 9)

In [24]:
product_main_category = []

for c in df['product_category_name']:
  product_main_category.append(switch(c))

df['product_main_category'] = product_main_category

#### 재구매 관련 파생변수

##### 재구매율 파생변수 : re_percent

In [26]:
group = df.groupby(['seller_id'])

In [28]:
d = group.customer_unique_id.nunique() # 스토어별 구매고객

In [29]:
group2 = df.groupby(['seller_id','customer_unique_id'])

In [30]:
a = (group2.nunique().order_id > 2).sum(level = 0) # 각 스토어별 재구매한 고객의 수

<ipython-input-30-5865e056b5b8>:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  a = (group2.nunique().order_id > 2).sum(level = 0) # 각 스토어별 재구매한 고객의 수


In [31]:
re = pd.DataFrame(a / d)

In [32]:
re.rename(columns = {0:'re_percent'}, inplace = True)

In [33]:
df = pd.merge(df,re, how='left',on='seller_id')

##### 재 구매 여부 파생변수 : re_ox

In [34]:
df['re_ox'] = 0
df.loc[df.re_percent != 0, 're_ox'] = 1 

In [35]:
df.re_ox.value_counts()

0    45243
1     2126
Name: re_ox, dtype: int64

#### 상품 설명 관련 파생변수

In [36]:
# 상품 정보 전달 정도를 파악할 수 있는 변수 : product_info

df['product_info'] = (df['product_photos_qty'] + df['product_description_lenght']) /2

#### 상품 크기 관련 파생변수

In [37]:
# 부피 변수 (사각형 부피 공식 사용-박스) : product_size

df['product_size'] = df['product_length_cm'] * df['product_width_cm'] * df['product_height_cm']

#### 판매자 관련 파생변수

참고 : https://www.kaggle.com/code/khakim17/custumer-satisfaction-prediction-and-analysist#CREATING-ANOTHER-COLUMNS-THAT-NEEDED

In [38]:
# 판매자 등급, 상품 등급, 특정 판매자가 판 상품 개수 변수 (SELLER RATING, PRODUCT RATING AND NUMBER OF PRODUCT SOLD BY SELLER)

In [39]:
seller_ratingx = {}
# product_ratingx = {}
product_soldx = {}

for item in df['seller_id'].unique():
  seller_ratingx[item] = round(df[df['seller_id'] == item].describe().iloc[1][3],1)
  product_soldx[item] = int(df[df['seller_id'] == item].describe().iloc[0][0])

# for item in df['product_id'].unique():
#   product_ratingx[item] = round(df[df['product_id'] == item].describe().iloc[1][3],1)
#   product_soldx[item] = int(df[df['seller_id'] == item].describe().iloc[0][0])

seller_rating = []
product_sold = []
for item in df['seller_id']:
  seller_rating.append(seller_ratingx[item])
  product_sold.append(product_soldx[item])

# product_rating = []
# for item in df['product_id']:
#   product_rating.append(product_ratingx[item])

df['seller_rating'] = seller_rating
df['product_sold'] = product_sold
# df['product_rating'] = product_rating

역시 시간이 오래 걸려서 seller_rating이랑 product_solidx 변수까지만 생성하는 걸로..!

#### 주문건수 관련 변수

##### 스토어 주문건수 변수 : seller_popularity

In [40]:
group = df.groupby(['seller_id'])

In [41]:
store_popularity = group.order_id.nunique()

In [42]:
store_popularity

seller_id
0015a82c2db000af6aaaf3ae2ecb0532     1
001cca7ae9ae17fb1caed9dfb1094831    88
002100f778ceb8431b7a1020ff7ab48f    15
004c9cd9d87a3c30c522c48c4fc07416    55
00720abe85ba0859807595bbf045a33b     5
                                    ..
ffc470761de7d0232558ba5e786e57b7    11
ffdd9f82b9a447f6f8d4b91554cc7dd3     9
ffeee66ac5d5a62fe688b9d26f83f534     8
fffd5413c0700ac820c7069d66d98c89    25
ffff564a4f9085cd26170f4732393726     6
Name: order_id, Length: 2385, dtype: int64

In [44]:
store_popularity = pd.DataFrame(store_popularity)
store_popularity.rename(columns = {'order_id':'seller_popularity'}, inplace = True)

In [45]:
store_popularity

,seller_popularity
seller_id,
0015a82c2db000af6aaaf3ae2ecb0532,1
001cca7ae9ae17fb1caed9dfb1094831,88
002100f778ceb8431b7a1020ff7ab48f,15
004c9cd9d87a3c30c522c48c4fc07416,55
00720abe85ba0859807595bbf045a33b,5
...,...
ffc470761de7d0232558ba5e786e57b7,11
ffdd9f82b9a447f6f8d4b91554cc7dd3,9
ffeee66ac5d5a62fe688b9d26f83f534,8


In [46]:
df = pd.merge(df,store_popularity, how='left',on='seller_id')

##### 고객 주문건수 변수 : 'cu_buy'

In [47]:
cu_group = df.groupby(['customer_unique_id'])

In [48]:
cu_buy = cu_group.order_id.nunique()

In [49]:
cu_buy = pd.DataFrame(cu_buy)
cu_buy.rename(columns = {'order_id':'cu_buy'}, inplace = True)

In [50]:
df = pd.merge(df,cu_buy, how='left',on='customer_unique_id')

#### 확인해보기

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47369 entries, 0 to 47368
Data columns (total 54 columns):
 #   Column                         Non-Null Count  Dtype          
---  ------                         --------------  -----          
 0   order_id                       47369 non-null  object         
 1   customer_id                    47369 non-null  object         
 2   order_status                   47369 non-null  object         
 3   order_purchase_timestamp       47369 non-null  datetime64[ns] 
 4   order_approved_at              47369 non-null  object         
 5   order_delivered_carrier_date   47369 non-null  object         
 6   order_delivered_customer_date  47369 non-null  datetime64[ns] 
 7   order_estimated_delivery_date  47369 non-null  datetime64[ns] 
 8   review_id                      47369 non-null  object         
 9   review_score                   47369 non-null  int64          
 10  review_comment_message         47369 non-null  object         
 11  re

In [54]:
df.loc[:,['seller_popularity','product_sold']]  # product_sold랑 seller_popularity랑 의도한 건 같은 것 같은데 값이 다르게 나온다...

,seller_popularity,product_sold
0,19,22
1,19,22
2,19,22
3,19,22
4,19,22
...,...,...
47364,1,1
47365,1,2
47366,1,2
47367,1,1


#### 리뷰 관련 파생변수

In [ ]:
# 리뷰 메세지 길이 변수(포르투갈ver) : review_comment_message_len
review_comment_message_len = []

for m in df['review_comment_message']:
  word = list(m.split())
  review_comment_message_len.append(len(word))

df['review_comment_message_len'] = review_comment_message_len

In [ ]:
#!pip install googletrans==4.0.0-rc1

In [ ]:
# 데이터 저장

#df.to_csv('/content/drive/MyDrive/BOAZ-Analysis/미니 프로젝트1/derived_variable.csv', index = False)

포르투갈어 -> 영어 번역 후 진행

In [ ]:
#포르투갈어 -> 영어 번역
import googletrans
import time
import random

translator = googletrans.Translator()
trans = []

for m in df['review_comment_message']:
  try:
    trans.append(translator.translate(m, src='pt', dest='en').text)
    time.sleep(1)
  except IndexError:
    trans.append('')

KeyboardInterrupt: ignored

In [ ]:
len(trans)

5300

너무 오래걸려서 ,,, 약 5천개까지만 번역

#### NLP

In [ ]:
df_nlp = df.copy()

In [ ]:
df_nlp = df_nlp.iloc[:5300, :]

In [ ]:
df_nlp['review_eng'] = trans

df_nlp.head(5)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,review_id,review_score,...,seller_state,delivery_period,early_delivery,early_delivery_div,income_level,product_size,product_info,review_comment_message_len,product_main_category,review_eng
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,a54f0611adc9ed256b57ede6b6eb5114,4,...,SP,-9 days +13:31:20,7 days 02:34:47,1,2,1976.0,136.0,32,2,"I haven't tested the product yet, but it came ..."
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,a54f0611adc9ed256b57ede6b6eb5114,4,...,SP,-9 days +13:31:20,7 days 02:34:47,1,2,1976.0,136.0,32,2,"I haven't tested the product yet, but it came ..."
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,a54f0611adc9ed256b57ede6b6eb5114,4,...,SP,-9 days +13:31:20,7 days 02:34:47,1,2,1976.0,136.0,32,2,"I haven't tested the product yet, but it came ..."
3,128e10d95713541c87cd1a2e48201934,a20e8105f23924cd00833fd87daa0831,delivered,2017-08-15 18:29:31,2017-08-15 20:05:16,2017-08-17 15:28:33,2017-08-18 14:44:43,2017-08-28,b46f1e34512b0f4c74a72398b03ca788,4,...,SP,-3 days +03:44:48,9 days 09:15:17,1,2,1976.0,136.0,15,2,They should pack the product better.The box ca...
4,0e7e841ddf8f8f2de2bad69267ecfbcf,26c7ac168e1433912a51b924fbd34d34,delivered,2017-08-02 18:24:47,2017-08-02 18:43:15,2017-08-04 17:35:43,2017-08-07 18:30:01,2017-08-15,dc90f19c2806f1abba9e72ad3c350073,5,...,SP,-6 days +23:54:46,7 days 05:29:59,1,2,1976.0,136.0,12,2,"I only found her small for six cups, but it's ..."


In [ ]:
# 영문자 외 공백으로 변환
import re

for r in df_nlp['review_eng']:
  r = re.sub('[^a-zA-Z]', ' ', r)

In [ ]:
# 대문자 => 소문자 변환 + split

low = []

for r in df_nlp['review_eng']:
  low.append(r.lower().split())

In [ ]:
df_nlp['review_split'] = low

In [ ]:
# 불용어 제거
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')


stops = set(stopwords.words('english')) # 영어 불용어 set
for words in df_nlp['review_split']:
  words=[w for w in words if not w in stops]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
